In [ ]:
from flask import Flask,request,jsonify
import subprocess
from tabulate import tabulate
import shlex
import os
import time
import yaml
import json
import asyncio

app = Flask(__name__)

@app.route('/')
def index():
    print("Root URL hitted")
    return 'Welcome to the app!'

##Create a new folder to hold function related files
def create_new_folder(baseDir,folder):
    #baseDir = "C:/Users/DeeJey/Downloads/openFaasFunctions"
    # Change the working directory
    os.chdir(baseDir)
    
    # mkdir command
    command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "mkdir {folder}"'
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=baseDir)
    # Wait for the command to finish and capture the output
    stdout, stderr = process.communicate()

    # Print the output
    print(stdout.decode())
    if stderr:
        print(stderr.decode())

##Create a new function
def create_new_function(baseDir,folder,function):
    dir = f'{baseDir}{folder}'
    # Change the working directory
    os.chdir(dir)

    # Run faas-cli new
    command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli new --lang python3 {function}"'
    
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Wait for the command to finish and capture the output
    stdout, stderr = process.communicate()

    # Print the output
    print(stdout.decode())
    if stderr:
        print(stderr.decode())

            
## add function content to created function
def over_ride_contents(new_content,baseDir,folder_name,function_name,filename):
    dir=f'{baseDir}{folder_name}/{function_name}'
    os.chdir(dir)
    
    with open(filename, "w") as f:
        f.write(new_content)
            

def edit_yml_file_imagename(dir,functionname,ymlname):
    os.chdir(dir)

    # Open the YAML file for reading
    with open(ymlname, 'r') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)

    # Modify the image value
    data['functions'][functionname]['image'] = f'dilax/{functionname}:latest'

    # Write the modified YAML file back to disk
    with open(ymlname, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
        
@app.route('/function/mem', methods=['GET'])
def edit_yml_file_mem_and_deploy():
    baseDir='C:/Users/DeeJey/Downloads/openFaasFunctions/'
    data = request.get_json()
    folder_name=data.get('folder')
    function_name = data.get('functionName')
    mem=data.get('mem')
    dir=f'{baseDir}{folder_name}'
    os.chdir(dir)
    ymlname=f'{function_name}.yml'
    
    # Open the YAML file for reading
    with open(ymlname, 'r') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)

    # Add the limits key if it doesn't exist
    if 'limits' not in data['functions'][function_name]:
        data['functions'][function_name]['limits'] = {}
    
    # Add the memory key if it doesn't exist
    if 'memory' not in data['functions'][function_name]['limits']:
        data['functions'][function_name]['limits']['memory'] = {}
    
    # Modify the mem
    data['functions'][function_name]['limits']['memory'] = f'{mem}'

    # Write the modified YAML file back to disk
    with open(ymlname, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
    
    deploy_function_private(baseDir,folder_name,function_name)
    
    response = {'message': 'Memory Modified successfully'}
    return jsonify(response)
    
##Build a function in privavte cloud
def build_function_in_private(baseDir,foldername,functionname):
    dir=f'{baseDir}{foldername}'
    yml=f'{functionname}.yml'
    
    os.chdir(dir)
    
    #Edit yaml befor further moves
    edit_yml_file_imagename(dir,functionname,yml)
    
    # Run faas-cli build
    command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli build -f {functionname}.yml"'
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Wait for the command to finish and capture the output
    stdout, stderr = process.communicate()

    # Print the output
    print(stdout.decode())
    if stderr:
        print(stderr.decode())
        
    update_to_dockerhub(dir,functionname)

def update_to_dockerhub(dir,functionname):
    os.chdir(dir)
   # command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "docker push dilax/fn42:latest"'
    command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli push -f {functionname}.yml"'
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Wait for the command to finish and capture the output
    stdout, stderr = process.communicate()

    # Print the output
    print(stdout.decode())
    if stderr:
        print(stderr.decode())
        

def deploy_function_private(baseDir,foldername,functionname):
    dir=f'{baseDir}{foldername}'
    os.chdir(dir)
    command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli deploy -f {functionname}.yml"'
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(stdout.decode())
    if stderr:
        print(stderr.decode())

    

## Import function and get output
@app.route('/function', methods=['GET'])
def deploy_function_main():
    baseDir='C:/Users/DeeJey/Downloads/openFaasFunctions/'
    data = request.get_json()
    folder_name=data.get('folder')
    function_name = data.get('functionName')
    content=data.get('function')
    
    ##private part
    create_new_folder(baseDir,folder_name)
    create_new_function(baseDir,folder_name,function_name)
    over_ride_contents(content,baseDir,folder_name,function_name,"handler.py")
    build_function_in_private(baseDir,folder_name,function_name)
    deploy_function_private(baseDir,folder_name,function_name)
        
    response = {'message': 'Data received successfully'}
    return jsonify(response)

def edit_yml_file_mem_and_deploy_local(baseDir,folder_name,function_name,mem):
    dir=f'{baseDir}{folder_name}'
    os.chdir(dir)
    ymlname=f'{function_name}.yml'
    
    # Open the YAML file for reading
    with open(ymlname, 'r') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)

    # Add the limits key if it doesn't exist
    if 'limits' not in data['functions'][function_name]:
        data['functions'][function_name]['limits'] = {}
    
    # Add the memory key if it doesn't exist
    if 'memory' not in data['functions'][function_name]['limits']:
        data['functions'][function_name]['limits']['memory'] = {}
    
    # Modify the mem
    data['functions'][function_name]['limits']['memory'] = f'{mem}'

    # Write the modified YAML file back to disk
    with open(ymlname, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
    
    deploy_function_private(baseDir,folder_name,function_name)

@app.route('/collectLogs', methods=['GET'])
def invoke_and_collect_logs_private():
    data = request.get_json()
    base_url='http://localhost:8080/function/'
    base_dir='C:/Users/DeeJey/Downloads/openFaasFunctions/'
    folder_name=data.get('folderName')
    function_name=data.get('functionName')
    mem_list=data.get('memList')
    logs=[]
    
    
    for mem in mem_list:
        #Re_deploy function with new memory
        edit_yml_file_mem_and_deploy_local(base_dir,folder_name,function_name,mem)
        invoketimes=data.get('invokeCount')
        start_time = time.time()
        
        while(invoketimes>0):
            print("inside while")
            command = f'curl {base_url}{function_name}'
            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
        #    print(stdout.decode())
            if stderr:
                 x=0
        #        print(stderr.decode())
                
            invoketimes=invoketimes-1
       
        end_time = time.time()
        intervalInSecond = end_time - start_time
      
    
    ##Collect Logs for 2 min
   # command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli logs {function_name} --since 2m"'
  #  process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  #  stdout, stderr = process.communicate()
  #  logs.append(stdout)
 #   if stderr:
 #       print(stderr.decode())
    print(logs)
    print(len(logs))
    #logs2 = [json.loads(log.decode('utf-8')) for log in logs] 
    return jsonify("dad")

@app.route('/logs',methods=['GET'])
def get_logs():
    logs=[]
    command = f'"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli logs fn42 --since 1h"'
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    logs.append(stdout)
    if stderr:
        print(stderr.decode())
    print(logs)
    print(len(logs))
    return jsonify("logs")
    
##Faas List to get details about the server
@app.route('/faas-list', methods=['GET'])
def get_faas_list():
    # Run faas-cli list as an administrator
    command = '"C:/Program Files/Git/bin/bash.exe" -l -c "faas-cli list"'
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Wait for the command to finish and capture the output
    stdout, stderr = process.communicate()

    # If there was an error, return an error message to the user
    if process.returncode != 0:
        error_message = stderr.decode('utf-8')
        response = {'message': f'Error running command: {error_message}'}
        return jsonify(response), 500

    # If the command ran successfully, return the output to the user
    output = stdout.decode('utf-8')
    table = [line.split() for line in output.strip().split('\n')[1:]]
    response = {'output': tabulate(table, headers=['Function', 'Invocations', 'Replicas'])}
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=False,port=9030)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9030/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2023 22:20:34] "GET /logs2 HTTP/1.1" 404 -
127.0.0.1 - - [27/Mar/2023 19:01:47] "GET /logs2 HTTP/1.1" 404 -
127.0.0.1 - - [28/Mar/2023 09:55:11] "GET / HTTP/1.1" 200 -


Root URL hitted


127.0.0.1 - - [28/Mar/2023 09:55:11] "GET /favicon.ico HTTP/1.1" 404 -
